# 关系数据库

基础的SQL学习

## Python DB-API

通过API包，使用Python控制数据库。

使用的模式大致如下：

先链接数据库（connect），这样就能得到一个Connection的实例，该实例可以：
- `.cursor()`，生成Cursor实例，即光标，可以进一步：
    - `.execute(SQL Code)`，执行SQL代码，然后
    - `.fetchone()`，获得第一行结果，或者
    - `.fetchall()`，获得所有结果
- 当执行一些修改代码（如插入、删除等），需要执行`.commit()`确认更改，或者
- 执行`.rollback()`回滚修改操作。

### SQL注入攻击

当我们在python中执行一些SQL查询时，可能会写如下的代码：
```python
c = connection.cursor()
c.execute("insert into posts values ('%s')" % content)
```

如上代码在执行普通查询时，并不会产生错误，但是如果插入的`content`中，包含`'`，就会报错

当插入的content为`'); delete from posts; --`时，会导致我们数据库`posts`中的全部数据被删除，这就是**SQL注入攻击**

我们可以修改代码为：
```python
c.execute("insert into posts values(%s)",(content,))
```
来避免SQL注入攻击。

**关键：在使用Python DB-API时，执行SQL代码不要使用输出格式化，如%，format或者f''string等**

### 脚本注入攻击

如果将content修改为：
```javascript
<script>
setTimeout(function() {
    var tt = document.getElementById('content');
    tt.value = "<h2 style='color: #FF6699; font-family: Comic Sans MS'>Spam, spam, spam, spam,<br>Wonderful spam, glorious spam!</h2>";
    tt.form.submit();
}, 2500);
</script>
```
也会发生错误，浏览器会不断输出spam（如上js代码的输出），这是因为，对于SQL来说，如上内容即字符串，但是当返回到浏览器中，进行解析时，浏览器会把它当作js并执行。

这并不是我们想要的结果，那该如何解决呢？

我们可以使用[Bleach](https://bleach.readthedocs.io/en/latest/)库，来帮我们对输入内容进行清理：
```python
# 安装bleach
pip install bleach
# 使用
import bleach

bleach.clean('an <script>evil()</script> example')
```
输出：`u'an &lt;script&gt;evil()&lt;/script&gt; example'`


如何处理数据库中已经存在的垃圾信息呢？
- 使用`UPDATE`，将垃圾信息替换为无害信息，比如说cheese!
    ```SQL
    UPDATE table
    SET column = value
    WHERE xxxxx;
    ```
- 使用`DELETE`，将垃圾信息删除
    ```SQL
    DELETE FROM table
    WHERE xxxxx;
    ```

## 数据库设计

要遵循规范式设计（Normalized Design）：
- 每一行都要有相同数量的列；
- 每张表中包含关键列（key），每一行都说明了该关键列的某些信息；
- 每张表中不需要包含说明非关键列信息的列，   
    比如：`姓名，年龄，购买商品，商品价格`中，`姓名`为关键列，而`商品价格`是对`购买商品`的说明，  
    所以，我们需要拆成两张表，即`姓名，年龄，购买商品`与`商品，商品价格`
- 表中的各列不应该有关系暗示，会产生误导，  
    比如：在员工技能表中
    <img src="https://s3.ax1x.com/2020/12/03/DTVuuR.png" alt="DTVuuR.png" border="0" width="300px"/>
    
    可能会误导，Annabel会Databases与English有关；会Linux与French有关。这时候，我们最好可以拆成两张表：
    
    <img src="https://s3.ax1x.com/2020/12/03/DTVRrn.md.png" alt="DTVRrn.png" border="0" width="500px"/>
    
